In [81]:
import os
import pandas as pd
import re
from datetime import datetime, timedelta
import json
from tqdm import tqdm

In [82]:
inputs_path = '../data/inputs-clean2.csv'
outputs_path = '../data/outputs.csv'

In [83]:
# Read the CSV files
df_in = pd.read_csv(inputs_path)
df_out = pd.read_csv(outputs_path)

print("Inputs columns:", df_in.columns.tolist())
print("Outputs columns:", df_out.columns.tolist())

/tmp/ipykernel_16394/3739053914.py:2: DtypeWarning: Columns (6,8,10,14,16,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df_in = pd.read_csv(inputs_path)


Inputs columns: ['id', 'date', 'time', 'A1', 'B1', 'C1', 'TitlePomembnoSLO', 'ContentPomembnoSLO', 'TitleNesreceSLO', 'ContentNesreceSLO', 'TitleZastojiSLO', 'ContentZastojiSLO', 'TitleVremeSLO', 'ContentVremeSLO', 'TitleOvireSLO', 'ContentOvireSLO', 'TitleDeloNaCestiSLO', 'ContentDeloNaCestiSLO', 'TitleOpozorilaSLO', 'ContentOpozorilaSLO', 'TitleMednarodneInformacijeSLO', 'ContentMednarodneInformacijeSLO', 'TitleSplosnoSLO', 'ContentSplosnoSLO']
Outputs columns: ['id', 'date', 'time', 'nujna', 'nova', 'programs', 'content', 'file_name', 'year', 'month']


In [84]:
# remove duplicate timestamps preffering NOVE prometne info
df_out['timestamp'] = pd.to_datetime(df_out['date'] + ' ' + df_out['time'], errors='coerce')

df_out['prefer_nove'] = df_out['content'].str.contains('NOVE|NOVA', case=False, na=False)

df_out_sorted = df_out.sort_values(by=['timestamp', 'prefer_nove'], ascending=[True, False])
df_out_dedup = df_out_sorted.drop_duplicates(subset='timestamp', keep='first')

df_out_dedup = df_out_dedup.drop(columns=['prefer_nove'])

df_out = df_out_dedup

print(df_out_dedup[['id', 'timestamp', 'content']])

                                         id           timestamp  \
0      f16895dd-6f45-50da-8ce6-966d44dbe3ef 2022-01-01 06:00:00   
1      6e542c9c-2559-5837-9c0b-23b4558fc710 2022-01-01 06:30:00   
2      376bf2cd-fd6a-51ff-bed5-ad53a48afe66 2022-01-01 07:00:00   
3      ba3714f5-35dc-539d-8c25-7da7f9846c64 2022-01-01 08:00:00   
4      3db9bb19-c329-5d58-a723-0394656d6ba5 2022-01-01 08:30:00   
...                                     ...                 ...   
27602  4cbae69e-1ccd-5a8f-97ef-eca72102c08c 2024-12-31 16:30:00   
27603  48a23f37-3823-532f-87e9-4656db1bba74 2024-12-31 17:00:00   
27604  88297f19-bb71-5c66-97d8-8c10c6d2fe0b 2024-12-31 17:30:00   
27605  c6baf039-a305-5acd-bb0d-beed3dd46ffa 2024-12-31 18:30:00   
27606  1bc35a0f-9597-52a5-8426-a75b49045332 2024-12-31 21:00:00   

                                                 content  
0      Prometne informacije       01. 01. 2022  \t   ...  
1      Prometne informacije       01. 01. 2022  \t   ...  
2      Prometne in

In [101]:
df_in['Datum'] = pd.to_datetime(df_in['date'] + ' ' + df_in['time'], errors='coerce')
df_out['row_dt'] = pd.to_datetime(df_out['date'] + ' ' + df_out['time'], errors='coerce')

title_content_cols = [
    'TitlePomembnoSLO', 'ContentPomembnoSLO',
    'TitleNesreceSLO', 'ContentNesreceSLO',
    'TitleZastojiSLO', 'ContentZastojiSLO',
    'TitleVremeSLO', 'ContentVremeSLO',
    'TitleOvireSLO', 'ContentOvireSLO',
    'TitleDeloNaCestiSLO', 'ContentDeloNaCestiSLO',
    'TitleOpozorilaSLO', 'ContentOpozorilaSLO',
    'TitleMednarodneInformacijeSLO', 'ContentMednarodneInformacijeSLO',
    'TitleSplosnoSLO', 'ContentSplosnoSLO'
]

def format_input_row(row):
    lines = []
    
    def add_spaces_to_title(title):
        return re.sub(r'(?<!^)(?=[A-Z])', ' ', title)

    for i in range(0, len(title_content_cols), 2):
        title_col = title_content_cols[i]
        content_col = title_content_cols[i + 1] if i + 1 < len(title_content_cols) else None
        
        if pd.notnull(row.get(content_col)):
            content = str(row[content_col]).strip()
            
            title = title_col.replace("Title", "").replace("SLO", "").strip()
            title = add_spaces_to_title(title)

            if pd.notnull(row.get(title_col)) and str(row[title_col]).strip():
                subtitle = str(row[title_col]).strip()
                subtitle = add_spaces_to_title(subtitle)
                title = f"{title} ({subtitle})"

            if title and content:
                lines.append(f"\t- **{title}**: {content}")
    
    return "\n".join(lines)


def clean_programs(programs):
    if isinstance(programs, str):
        if re.match(r"^\d{2},", programs):
            return programs[3:]
    return programs

    
def generate_input_output_pair(ix):
    row = df_out.loc[ix]
    row_dt = row['row_dt']

    # Adjust the time window as needed
    start_window = row_dt - timedelta(hours=1)
    end_window = row_dt - timedelta(minutes=1)

    df_in_window = df_in[(df_in['Datum'] >= start_window) & (df_in['Datum'] < end_window)].copy()
    df_in_window = df_in_window.sort_values(by='Datum')

    input_texts = []
    for _, input_row in df_in_window.iterrows():
        formatted_block = format_input_row(input_row)
        if formatted_block.strip():
            timestamp_str = input_row['Datum'].strftime('%d. %m. %y %H.%M')
            input_texts.append(f"- {timestamp_str} :\n{formatted_block}")

    combined_input = "\n\n".join(input_texts)

    return {
        'timestamp': row_dt,
        'input': combined_input,
        'output': row['content'],
        'nujna': row['nujna'],
        'programs': clean_programs(row['programs']),
    }

input_output_df = pd.DataFrame([
    generate_input_output_pair(ix) for ix in tqdm(df_out.index, desc="Generating input/output pairs")
])

# Preview
print(input_output_df.head())


enerating input/output pairs: 100%|█████████████████████████| 24905/24905 [00:53<00:00, 464.50it/s]

            timestamp                                              input  \
0 2022-01-01 06:00:00  - 01. 01. 22 05.15 :\n\t- **Vreme**: Ponekod p...   
1 2022-01-01 06:30:00  - 01. 01. 22 05.33 :\n\t- **Vreme**: Ponekod p...   
2 2022-01-01 07:00:00  - 01. 01. 22 06.31 :\n\t- **Vreme**: Ponekod p...   
3 2022-01-01 08:00:00  - 01. 01. 22 07.21 :\n\t- **Vreme**: Ponekod p...   
4 2022-01-01 08:30:00  - 01. 01. 22 08.00 :\n\t- **Vreme**: Ponekod p...   

                                              output  nujna programs  
0  Prometne informacije       01. 01. 2022  \t   ...      0      1,2  
1  Prometne informacije       01. 01. 2022  \t   ...      0      1,2  
2  Prometne informacije       01. 01. 2022  \t   ...      0      1,2  
3  Prometne informacije       01. 01. 2022  \t   ...      0      1,3  
4  Prometne informacije       01. 01. 2022  \t   ...      0        2  


In [102]:
example = input_output_df[input_output_df['nujna'] == False].iloc[48]

print("=== Timestamp ===")
print(example['timestamp'])

print("\n=== Programs ===")
print(example['programs'])

print("\n=== Input ===")
print(example['input'])

print("\n=== Output ===")
print(example['output'])

=== Timestamp ===
2022-01-03 06:30:00

=== Programs ===
1,2

=== Input ===
- 03. 01. 22 05.30 :
	- **Zastoji**: Zastoj je na cesti Pijava Gorica - Škofljica.
	- **Vreme**: Megla v pasovih ponekod po Sloveniji zmanjšuje vidljivost.
	- **Opozorila (Omejitve za tovorna vozila)**: Od 30. decembra je v veljavi sprememba omejitve za tovorna vozila nad 7,5 ton. Več.
	- **Mednarodne Informacije**: Čakalna doba je na Gruškovju.

- 03. 01. 22 05.36 :
	- **Zastoji**: Na dolenjski avtocesti od Grosuplja proti Ljubljani. Na cesti Pijava Gorica - Škofljica.
	- **Vreme**: Megla v pasovih ponekod po Sloveniji zmanjšuje vidljivost.
	- **Opozorila (Omejitve za tovorna vozila)**: Od 30. decembra je v veljavi sprememba omejitve za tovorna vozila nad 7,5 ton. Več.
	- **Mednarodne Informacije**: Čakalna doba je na Gruškovju.

- 03. 01. 22 05.45 :
	- **Zastoji**: Na dolenjski avtocesti od Grosuplja proti Ljubljani. Na cesti Pijava Gorica - Škofljica.
	- **Vreme**: Megla v pasovih ponekod po Sloveniji zmanjšu

In [103]:
latest_timestamp = input_output_df['timestamp'].max()

cutoff_timestamp = latest_timestamp - pd.DateOffset(months=4)

train_input_output_df = input_output_df[input_output_df['timestamp'] < cutoff_timestamp]
test_input_output_df = input_output_df[input_output_df['timestamp'] >= cutoff_timestamp]

print(f"Train set size: {len(train_input_output_df)} records")
print(f"Test set size: {len(test_input_output_df)} records")

Train set size: 22310 records
Test set size: 2595 records


In [104]:
train_input_output_df.to_csv("../data/trainset.csv", index=False)
test_input_output_df.to_csv("../data/testset.csv", index=False)

In [105]:
empty_counts = {
    col: (train_input_output_df[col].astype(str).str.strip() == '').sum()
    for col in ['input', 'output', 'timestamp']
}

print("Empty counts:")
print(empty_counts)

empty_input_rows = test_input_output_df[
    test_input_output_df['input'].astype(str).str.strip() == ''
]

print("Timestamps of rows with empty input:")
print(empty_input_rows[['timestamp']])

for i in range(2):
    print(f"\n--- Example {i+1} ---\n")
    print(test_input_output_df['input'].dropna().iloc[i])

Empty counts:
{'input': np.int64(465), 'output': np.int64(0), 'timestamp': np.int64(0)}
Timestamps of rows with empty input:
                timestamp
22369 2024-09-03 18:30:00
22456 2024-09-08 07:00:00
22545 2024-09-11 21:00:00
22641 2024-09-16 11:30:00
22811 2024-09-24 17:30:00
...                   ...
24888 2024-12-31 08:30:00
24889 2024-12-31 09:00:00
24890 2024-12-31 09:30:00
24896 2024-12-31 12:30:00
24897 2024-12-31 13:00:00

[71 rows x 1 columns]

--- Example 1 ---

- 31. 08. 24 21.27 :
	- **Zastoji**: Na gorenjski avtocesti pred predorom Karavanke proti Avstriji, 1 km.Na štajerski avtocesti pred prehodom Šentilj proti Avstriji, 1 km.
	- **Ovire**: Na gorenjski avtocesti je zaradi okvare vozila zaprt prehitevalni pas pred izvozom Jesenice vzhod v smeri Karavank.
	- **Delo Na Cesti**: V ponedeljek, 2. 9., med 14.30 in 20.30, bo promet skozi predor Karavanke potekal izmenično enosmerno s čakanjem pred predorom.Cesta Litija - Zagorje danes ne bo zaprta pri Šklendrovcu.Cesta Ljubl